In [28]:
import numpy as np
import matplotlib.pyplot as plt
from rich.table import Table
from rich import print as rptint
import json
import os
import pandas as pd
from string import digits

scenario_name = "CWE-89-SQL-injection"
TIME_PERIOD = 1
TIME_INTERVAL = '1MS'
save_file_path = os.path.join('K:/hids/LID-DS-master/tools/out', scenario_name)

,syscall,DIR,RET,ARGS1
0,write,>,fd=5(<f>/service/upload/image_0456.eps),size=15101


获取数据

In [2]:
test_normal_df = pd.read_pickle(os.path.join(save_file_path,  'DF_test_normal.pkl'))
test_exploit_df = pd.read_pickle(os.path.join(save_file_path, 'DF_test_exploit.pkl'))

In [15]:
has_arg_df = test_normal_df[test_normal_df['ARGS'].isna() == False]

In [19]:
has_arg_df

,time,UserID,PID,ProcessName,TID,syscall,DIR,ARGS
0,1631083490806378308,101,3861861,mysqld,3862354,select,<,res=0
2,1631083490806627721,101,3861861,mysqld,3862355,select,<,res=0
4,1631083490812467214,101,3861861,mysqld,3862357,select,<,res=0
26,1631083491296433028,33,3862925,apache2,3862925,poll,<,res=1
27,1631083491296435798,33,3862925,apache2,3862925,read,>,fd=10(<4t>192.168.128.2:36280->192.168.128.10:80)
...,...,...,...,...,...,...,...,...
3857206,1631054137372659043,33,3256522,apache2,3256522,writev,<,res=2461
3857207,1631054137372663744,33,3256522,apache2,3256522,write,>,fd=7(<f>/var/log/apache2/access.log)
3857208,1631054137372668967,33,3256522,apache2,3256522,write,<,res=257
3857211,1631054137372673413,33,3256522,apache2,3256522,poll,>,fds=10:41


In [40]:

def process_path(file_path_i):
    PATH_LENGTH = 3
    print(f'file_path_i {file_path_i}')
    if "(" in file_path_i and ")" in file_path_i:
        file_path_between_par = file_path_i[file_path_i.find("(") + 1:file_path_i.find(")")]
        file_path_before_par = file_path_i[:file_path_i.find("(")]
        file_path_max = max([file_path_between_par, file_path_before_par], key=len)

        if file_path_i.count("/") > PATH_LENGTH:
            file_path_min = min([file_path_between_par, file_path_before_par], key=len)
            file_path_i = file_path_max.replace(file_path_min, "")
        else:
            file_path_i = file_path_max

    else:
        if file_path_i.count("/") > PATH_LENGTH:
            full_path = urlparse(file_path_i)
            file_path_i = os.path.dirname(full_path.path)

    processed_file_path = file_path_i.rsplit('/')
    path_last_element = processed_file_path[-1]

    if '>' in processed_file_path[0]:
        processed_file_path[0] = ''

    # we clean the path from random characters and numbers
    if path_last_element.islower():
        processed_file_path[-1] = path_last_element.translate({ord(k): None for k in digits})
        processed_file_paths = '/'.join(processed_file_path)
    else:
        processed_file_paths = '/'.join(processed_file_path[:-1])

    print(f'processed_file_paths {processed_file_paths}')
    return processed_file_paths

getfile_from_in_ret_list = ['write', 'writev', 'pwrite', 'pread', 'close', 'getdents64', 'getdents', 'ioctl', 'execve', 'lseek', 'fcntl', 'dup', 'flock', 'fstat']
getfile_from_out_ret_list = ['openat']
getfile_from_arg1_list = ['lstat', 'chmod', 'open', 'clone', 'vfork', 'access', 'stat', 'unlink', 'getcwd', 'mkdir', 'rename', 'chdir']

def get_filepath(row):
    syscall = row['syscall']
    if row['DIR'] == '>':
        if syscall in getfile_from_in_ret_list:
            _, file_path = row['RET'].split('=')
        elif syscall == 'sendfile':
            _, file_path = row['ARGS1'].split('=')
        elif syscall == 'mmap':
            _, file_path = row['ARGS4'].split('=')
        else:
            return np.nan
    else:
        if syscall == 'openat':
            _, file_path = row['RET'].split('=')
        elif syscall in getfile_from_arg1_list:
            _, file_path = row['ARGS1'].split('=')
        else:
            return np.nan

    return process_path(file_path)

row = pd.DataFrame(columns=['syscall', 'DIR', 'RET', 'ARGS1'])
row.loc[len(row.index)] = ['write','>','fd=5(<f>/service/upload/image_0456.eps)','size=15101']
row.loc[len(row.index)] = ['sendfile','>','out_fd=3(<4t>172.24.0.6:48956->172.24.0.7:80)','in_fd=10(<f>/etc/nginx/html/index.html)']
row.loc[len(row.index)] = ['openat','<','fd=46(<d>/usr/src/blog/db/migrate)','dirfd=-100(AT_FDCWD)']
row.loc[len(row.index)] = ['lstat','<','res=0','path=/usr']
row.apply(get_filepath, axis=1, result_type='expand')

file_path_i 5(<f>/service/upload/image_0456.eps)
processed_file_paths /service/upload/image_.eps
file_path_i 10(<f>/etc/nginx/html/index.html)
processed_file_paths /etc/nginx/html/index.html
file_path_i 46(<d>/usr/src/blog/db/migrate)
processed_file_paths /usr/src/blog/db/migrate
file_path_i /usr
processed_file_paths /usr


0    /service/upload/image_.eps
1    /etc/nginx/html/index.html
2      /usr/src/blog/db/migrate
3                          /usr
dtype: object